# Data Exploration with Spark

Dr Jose M. Albornoz, April 2019

In this notebook a few data exploration techniques will be examined

In [1]:
import org.apache.spark.sql.types._
import org.apache.spark.sql.Row

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.192:4040
SparkContext available as 'sc' (version = 2.4.0, master = local[*], app id = local-1556212296140)
SparkSession available as 'spark'


import org.apache.spark.sql.types._
import org.apache.spark.sql.Row


# 1.- Load data

In [2]:
val rawDF = spark.read.format("csv").option("delimiter"," ").option("inferSchema","true").load("Data/LifeExpectancy.txt")

rawDF: org.apache.spark.sql.DataFrame = [_c0: string, _c1: string ... 3 more fields]


In [3]:
rawDF.count

res0: Long = 197


In [4]:
rawDF.show(5)

+-----------+----+------+----+----+
|        _c0| _c1|   _c2| _c3| _c4|
+-----------+----+------+----+----+
|Afghanistan|null|48.673|null| SAs|
|    Albania|null|76.918|null|EuCA|
|    Algeria|null|73.131|null|MENA|
|     Angola|null|51.093|null| SSA|
|  Argentina|null|75.901|null|Amer|
+-----------+----+------+----+----+
only showing top 5 rows



In [5]:
val schema = StructType(Array(
       StructField("Country",StringType),
       StructField("LifeExp",DoubleType),
       StructField("Region",StringType)
))

schema: org.apache.spark.sql.types.StructType = StructType(StructField(Country,StringType,true), StructField(LifeExp,DoubleType,true), StructField(Region,StringType,true))


In [6]:
val selectedDF = rawDF.select("_c0","_c2","_c4")
val lifeExpectancyDF = spark.createDataFrame(selectedDF.rdd,schema)

selectedDF: org.apache.spark.sql.DataFrame = [_c0: string, _c2: double ... 1 more field]
lifeExpectancyDF: org.apache.spark.sql.DataFrame = [Country: string, LifeExp: double ... 1 more field]


In [7]:
lifeExpectancyDF.show(5)

+-----------+-------+------+
|    Country|LifeExp|Region|
+-----------+-------+------+
|Afghanistan| 48.673|   SAs|
|    Albania| 76.918|  EuCA|
|    Algeria| 73.131|  MENA|
|     Angola| 51.093|   SSA|
|  Argentina| 75.901|  Amer|
+-----------+-------+------+
only showing top 5 rows



# 2.- Five numbers summary

The Five Number Summary Contains refers to the following statistics:

* Min - Minimum value of the column
* First Quantile - 25% percentile of the column
* Median
* Third Quartile - 75%th percentile of the column
* Max - maximum value of the column

In [8]:
lifeExpectancyDF.describe("LifeExp").show()

+-------+-----------------+
|summary|          LifeExp|
+-------+-----------------+
|  count|              197|
|   mean|69.86281725888323|
| stddev|9.668736205594511|
|    min|           47.794|
|    max|           83.394|
+-------+-----------------+



Instead of calculating quantiles values and median, Spark's *describe* computes mean and standard deviation. The reason for this is that median and quantiles are costly to compute on large data; the data must be sorted order and the result of the calculation is skewed.

We can calculate quartiles using the *approxQuantile* method introduced in Spark 2.0. The name of the method suggests that it provides approximate values, whose precision depend on a specified error rate; this makes calculation faster.

In [9]:
val medianAndQuantiles = lifeExpectancyDF.stat.approxQuantile("LifeExp", Array(0.25, 0.5, 0.75),0.0)

medianAndQuantiles: Array[Double] = Array(64.666, 73.235, 76.652)


# 3.- Histograms

Spark's Dataframe API doesn’t have a built-in function for histograms; however such a functionality exists in the RDD API.

In [10]:
val (startValues, counts) = lifeExpectancyDF.select("LifeExp").map(value => value.getDouble(0)).rdd.histogram(5)

startValues: Array[Double] = Array(47.794, 54.914, 62.034, 69.154, 76.274, 83.394)
counts: Array[Long] = Array(24, 18, 32, 69, 54)


The result of the histogram is two arrays:

* The first array contains the starting values of each bin
* The second array contains the count for each bin
    
We can thus see, for example, that there are 24 countries with a life expectancy between 47.8 and 54.9 years.  

Our next step is to transform these arrays into a dataframe and then into a SparkSQL table.

In [11]:
val zippedValues = startValues.zip(counts)
case class HistRow(startPoint:Double, count:Long)
val rowRDD = zippedValues.map(value => HistRow(value._1, value._2))
val histDF = spark.createDataFrame(rowRDD)
histDF.createOrReplaceTempView("histogramTable")

zippedValues: Array[(Double, Long)] = Array((47.794,24), (54.914,18), (62.034,32), (69.154,69), (76.274,54))
defined class HistRow
rowRDD: Array[HistRow] = Array(HistRow(47.794,24), HistRow(54.914,18), HistRow(62.034,32), HistRow(69.154,69), HistRow(76.274,54))
histDF: org.apache.spark.sql.DataFrame = [startPoint: double, count: bigint]


In [12]:
histDF.show

+----------+-----+
|startPoint|count|
+----------+-----+
|    47.794|   24|
|    54.914|   18|
|    62.034|   32|
|    69.154|   69|
|    76.274|   54|
+----------+-----+



In Apache Zeppelin plotting the histogram is a straightforward task once we have a SparkSQL table like **histogramTable**. Unfortunately this is not as simple in Jupyter!

# 4.- Outlier detection through the outlier labeling method

The outlier labeling method is based on finding deviations with respect to the interquartile range (IQR):

IQR = Q3 - Q1

where Q3 and Q1 are the 75th and 25th quantiles respectively. In this way, any value smaller than Q1- 1.5 * IQR or any value greater than Q3+1.5 * IQR will be categorised as an outlier.

We will first create some dummy data to illustrate the method:

In [13]:
val sampleData = List(10.2, 14.1, 14.4, 14.4, 14.4, 14.5, 14.5, 14.6, 14.7, 14.7, 14.7, 14.9, 15.1, 15.9, 16.4)
   
val rowRDD = spark.sparkContext.makeRDD(sampleData.map(value => Row(value)))

sampleData: List[Double] = List(10.2, 14.1, 14.4, 14.4, 14.4, 14.5, 14.5, 14.6, 14.7, 14.7, 14.7, 14.9, 15.1, 15.9, 16.4)
rowRDD: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = ParallelCollectionRDD[43] at makeRDD at <console>:30


In [14]:
val schema = StructType(Array(StructField("value", DoubleType)))

val df = spark.createDataFrame(rowRDD, schema)

schema: org.apache.spark.sql.types.StructType = StructType(StructField(value,DoubleType,true))
df: org.apache.spark.sql.DataFrame = [value: double]


We now calculate the required quantiles and the IQR:

In [15]:
val quantiles = df.stat.approxQuantile("value", Array(0.25,0.75),0.0)
val Q1 = quantiles(0)
val Q3 = quantiles(1)
val IQR = Q3 - Q1

quantiles: Array[Double] = Array(14.4, 14.9)
Q1: Double = 14.4
Q3: Double = 14.9
IQR: Double = 0.5


So we can now filter the outliers:

In [16]:
val lowerRange = Q1 - 1.5*IQR
val upperRange = Q3 +  1.5*IQR

val outliers = df.filter(s"value < $lowerRange or value > $upperRange")
outliers.show()

+-----+
|value|
+-----+
| 10.2|
| 15.9|
| 16.4|
+-----+



lowerRange: Double = 13.65
upperRange: Double = 15.65
outliers: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [value: double]
